Gans (408)

# Upload files

In [1]:
from google.colab import files


uploaded = files.upload()


Saving antigen_antibody.csv to antigen_antibody.csv


# Import stuff


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorboard
import datetime
import io
from tensorflow import keras
from sklearn.model_selection import train_test_split
from numpy import unique
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers.merge import concatenate
from sklearn.preprocessing import StandardScaler

## Preprocess Labels

In [3]:
def preprocess_labels(label):
  for i in range(0, len(label)):
    label.iloc[i] = 1.0 if label.iloc[i]=='Y' else 0.0
  label = np.asarray(label).astype(float)
  label = tf.cast(label, dtype=tf.int32)
  return label

## Prepare Inputs Encoded

In [4]:
def prepare_inputs(x_vals):
  x_vals_enc = list()
  for i in range(x_vals.shape[1]):
    le = LabelEncoder()
    le.fit(x_vals[:, i])
    x_enc = le.transform(x_vals[:, i])
    x_vals_enc.append(x_enc)
  return x_vals_enc

## Random Shit

In [ ]:
x = tf.constant([[[0]]] * 3 + [[[1]]] * 2)
x

<tf.Tensor: shape=(5, 1, 1), dtype=int32, numpy=
array([[[0]],

       [[0]],

       [[0]],

       [[1]],

       [[1]]], dtype=int32)>

In [ ]:
x = tf.Variable([[1, 2, 3],
     [4, 5, 6]])
x[:, -1].numpy()     

array([3, 6], dtype=int32)

In [ ]:
t = tf.Variable([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])
print(t)
t = t[np.newaxis, :, :]
t

<tf.Variable 'Variable:0' shape=(3, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]], dtype=int32)>


<tf.Tensor: shape=(1, 3, 3), dtype=int32, numpy=
array([[[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]], dtype=int32)>

## Callbacks

In [5]:
# Callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_model.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                  restore_best_weights=True)

# Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
!rm -rf ./logs/ #Clear logs

# Preprocessing

In [6]:
# Read, DataFrame it, and shuffle it.
df = pd.read_csv("/content/antigen_antibody.csv", encoding= 'unicode_escape')
df = pd.DataFrame(df)
df = df.sample(frac=1)
df = df.dropna(axis=1)
labels = df["Docking"]
df_num = df[['Hydro_L1', 'pl_L1', 'Hydro_L2', 'pl_L2',
       'Hydro_L3', 'pl_L3', 'Hydro_H1', 'pl_H1', 'Hydro_H2', 'pl_H2',
       'Hydro_H3', 'pl_H3']]
df_cat = df[['Seq_Hchain', 'H_CDR1', 'H_CDR2', 'H_CDR3', 'L_CDR1', 'L_CDR2', 'L_CDR3']]   
cat_enc = pd.DataFrame(prepare_inputs(df_cat.values)).T
df = pd.DataFrame(pd.concat([df_num, cat_enc], axis=1))

x_train_full, x_test, y_train_full, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.5, random_state=42)

print(x_train.columns)



Index(['Hydro_L1',    'pl_L1', 'Hydro_L2',    'pl_L2', 'Hydro_L3',    'pl_L3',
       'Hydro_H1',    'pl_H1', 'Hydro_H2',    'pl_H2', 'Hydro_H3',    'pl_H3',
                0,          1,          2,          3,          4,          5,
                6],
      dtype='object')


In [7]:
def go(x_vals):
  df = pd.DataFrame(x_vals)
  df_num = df[['Hydro_L1', 'pl_L1', 'Hydro_L2', 'pl_L2',
       'Hydro_L3', 'pl_L3', 'Hydro_H1', 'pl_H1', 'Hydro_H2', 'pl_H2',
       'Hydro_H3', 'pl_H3']]
  #df_cat = df[['Seq_Hchain', 'H_CDR1', 'H_CDR2', 'H_CDR3', 'L_CDR1', 'L_CDR2', 'L_CDR3']]       
  df_cat = df[[0, 1, 2, 3, 4, 5, 6]]       

  # Standardize numerical columns
  scaler = StandardScaler()
  df_num = scaler.fit_transform(df_num)
  return df_num, df_cat

df_num_train, df_cat_train = go(x_train)
df_num_valid, df_cat_valid = go(x_valid)
df_num_test, df_cat_test = go(x_test)  

In [8]:
# Make train, valid, and test
y_train = preprocess_labels(y_train)
y_valid = preprocess_labels(y_valid)
y_test = preprocess_labels(y_test)


# Building Model

In [9]:
def get_model(df, categorical_columns):
  cat_inputs, outputs = [], []

  for c in categorical_columns:
    num_unique_values = int(df[c].nunique())
    embed_dim = int(min(np.ceil(num_unique_values / 2), 60))
    inp = tf.keras.layers.Input(shape=(1,))
    out = tf.keras.layers.Embedding(num_unique_values + 1, embed_dim, embeddings_regularizer=tf.keras.regularizers.l1(0.0006))(inp)
    out = tf.keras.layers.Reshape(target_shape=(embed_dim,))(out)
    cat_inputs.append(inp)
    outputs.append(out)
  
  numerical_inputs = tf.keras.layers.Input(shape=(12,))
  x = tf.keras.layers.Concatenate()(outputs)
  x = tf.keras.layers.Concatenate()([numerical_inputs, x])
  x = x[np.newaxis, :, :]
  for rate in (1, 2, 4, 8) * 2:
    x = keras.layers.Conv1D(filters=20, kernel_size=20, padding="causal",
                                activation="relu", dilation_rate=rate)(x)
    x = keras.layers.MaxPool1D()(x)                                
  x = keras.layers.Conv1D(filters=10, kernel_size=1)(x)
  x = tf.keras.layers.Dropout(0.3)(x)
  y = tf.keras.layers.Dense(1, activation="sigmoid")(x)
  model = tf.keras.Model(inputs=[numerical_inputs, cat_inputs], outputs=y)

  model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9, nesterov=True, clipnorm=1.),
                metrics=["accuracy"])

  return model

# Fit and Evaluate

In [10]:
features = [f for f in df.columns if f not in ['Hydro_L1', 'pl_L1', 'Hydro_L2', 'pl_L2',
                                              'Hydro_L3', 'pl_L3', 'Hydro_H1', 'pl_H1', 'Hydro_H2', 'pl_H2',
                                              'Hydro_H3', 'pl_H3']]
model = get_model(df, features)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
df_cat_train = pd.DataFrame(df_cat_train)
df_cat_valid = pd.DataFrame(df_cat_valid)
ctrain = [df_cat_train.loc[:, f].values for f in features]
cvalid = [df_cat_valid.loc[:, f].values for f in features]
model.fit([df_num_train, ctrain], y_train, epochs=1000,
          validation_data=([df_num_valid, cvalid], y_valid)) 

Epoch 1/1000


In [ ]:
df_cat_test = pd.DataFrame(df_cat_test)
model.evaluate([df_num_test, [df_cat_test.loc[:, f].values for f in features]], y_test)

# Best is currently 80%